# The Convolution Theorem

In [1]:
%run ../setup.ipynb

Recall from {eq}`convolution` the definition of **convolution** between a signal $x$ (of length $N$) and impulse response $h$ (of length $K$):

```{math}
y[n] = (h * x)[n] = \sum_{k=0}^{K-1} h[k] \cdot x[n-k].
```

As we saw previously, this can be interpreted as constructing the output $y$ by mixing together several delayed copies of $x$ (index $k$ corresponds to the delay in samples), each with a gain coefficient given by $h[k]$.

In this section, our goal is to understand the frequency domain representation $Y = \text{DFT}(h * x)$ in terms of the DFTs of the inputs $h$ and $x$, which will be expressed succinctly by the **convolution theorem**.

## Circular convolution

Before we can properly apply the DFT to convolution, we'll first need to deal with the differing assumptions of how negative sample indices are handled.
When we first saw convolution, it was defined using the assumption that $x[n-k] = 0$ if $n < k$, i.e., that the signal $x$ is silent before the 0th sample.
Different ways of interpreting this assumption gave rise to the different {ref}`convolution modes <convolution-modes>` (full, valid, same).

The DFT, on the other hand, assumes that signals *repeat* indefinitely, so that $x[n-k] = x[n-k + N]$.

If we define convolution using the repetition assumption, we get what is known as **circular convolution**.
The equation is exactly the same as {eq}`convolution`; all that has changed is the interpretation of negative sample indices, which now wrap around to the end of the signal.

## The convolution theorem

Now that we've defined circular convolution, we can formally state the **convolution theorem**, which is one of the most important theorems in signal processing!

````{margin} 
```{tip}
We have assumed for convenience that both $h$ and $x$ have the same length $N$.  In practice, if one is shorter, we can equivalently pad it with zeros to match the length $N$ of the longer input.

This will not change the behavior of the convolution, but it does ensure that the DFTs have the same number of components.
```
````

````{admonition} Theorem: convolution in the frequency domain
Let $h$ and $x$ be sequences of length $N$, and let $y=h*x$ denote the circular convolution between them.

The DFT of the convolution is the product of the DFTs:

```{math}
:label: eq-conv-thm
y = h * x \quad \quad \Leftrightarrow \quad\quad Y[m] = H[m] \cdot X[m].
```
````

### Proof

By definition, the output signal $y$ is a sum of delayed copies of the input $x[n-k]$, each scaled by the corresponding coefficient $h[k]$.

By the {ref}`DFT linearity theorem <dft-linearity>`, we can think of the DFT $Y$ as a weighted combination of DFTs:

```{math}
\begin{align*}
Y = \text{DFT}(y) &= \text{DFT}\left(\sum_{k=0}^{N-1} h[k] \cdot x[n-k]\right) & \text{By definition of } y\\
&= \sum_{k=0}^{N-1} h[k] \cdot \text{DFT}(x[n-k]) & \text{By DFT linearity}.
\end{align*}
```
To reduce clutter, let's define $X_k = \text{DFT}(x[n-k])$ to be the DFT of the $k$-step delayed copy of $x$.

Now, we can use the {ref}`DFT Shifting Theorem <shifting-theorem>` to express the DFT of the delayed signal $X_k$ in terms of the original signal $X = \text{DFT}(x)$ for each frequency index $m$:

$$
X_k[m] = X[m] \cdot e^{-2\pi \cdot \mathrm{j} \cdot m \cdot k / N}.
$$

Substituting this into our derivation for $Y$, we can continue:

$$
\begin{align*}
Y[m] &= \sum_{k=0}^{N-1} h[k] \cdot X_k[m]\\
&= \sum_{k=0}^{N-1} h[k] \cdot X[m] \cdot e^{-2\pi \cdot \mathrm{j} \cdot m \cdot k / N}  & \text{DFT Shifting theorem}\\
&= X[m] \cdot \sum_{k=0}^{N-1} h[k] \cdot e^{-2\pi \cdot \mathrm{j} \cdot m \cdot k / N} & X[m] \text{ does not depend on } k\\
&= X[m] \cdot H[m] & \text{Definition of DFT} \\
&= H[m] \cdot X[m] & \text{By commutativity: } w \cdot z = z \cdot w.\\
\end{align*}
$$

which is exactly what we set out to prove.


## Consequences of the convolution theorem

Now that we have the convolution theorem, let's take some time to explore what it gives us.

### Fast convolution

From the convolution theorem, we get $Y[m] = H[m] \cdot X[m]$.
Applying the inverse DFT, we can recover the time-domain output signal:

```{math}
y[n] = \text{IDFT}(H \cdot X)
```
where the product $H\cdot X$ is the element-wise product of $H$ and $X$.

This gives us a recipe for computing the convolution using the DFT:

1. Pad $x$ and $h$ to the same length (if necessary)
2. Compute DFTs $X$ and $H$,
3. Multiply $Y[m] = H[m] \cdot X[m]$,
4. Take the inverse DFT of $Y$.

If $h$ and $x$ are of comparable length (say, both $N$ samples), then this can be more efficient than the {ref}`direct convolution algorithm <direct-convolution>`.
The direct convolution has an outer loop of $N$ steps (one for each output sample) and an inner loop of $N$ steps (one for each delay), for a total of $\sim N^2$ computation.

However, the recipe above takes:

1. $\leq N$ steps for padding
2. Two DFT calculations
3. $N$ steps to multiply the spectra
4. One inverse DFT calculation

Steps 2 and 4 can be performed with $\sim N + N \cdot \log N$ steps.
The total computation time will therefore scale like $N\cdot \log N \ll N^2$!

However, if $h$ is much shorter than $N$, this may not be worth it, as the direct method would take $N\cdot K$ steps.
Standard convolution implementations like `scipy.signal.convolve` typically compare the lengths of the signal to determine the most efficient means of computing the result.

### Algebraic properties

You may recall from the earlier section on {ref}`properties of convolution <convolution-properties>` that we asserted (without proof) that convolution is commutative:

```{math}
h*x = x*h,
```

and associative:

```{math}
h*(g*x) = (h*g)*x,
```

but we did not prove those properties.  (We could have, but it would have been pretty tedious!)

The convolution theorem provides a more direct and intuitive way to see these properties:

- Commutativity must exist because complex multiplication is commutative: $X[m]\cdot H[m] = H[m] \cdot X[m]$.
- Likewise for associativity: $H[m] \cdot (G[m] \cdot X[m]) = (H[m] \cdot G[m]) \cdot X[m]$.

Inheriting these properties from multiplication is much easier than deriving them from scratch!

### Filter analysis

Finally, the convolution theorem provides a way to understand the effect a particular impulse response $h$ might have on a signal.
Initially, we thought of each $h[k]$ as the gain applied to the delayed signal $x[n-k]$.

In the frequency domain, each $H[m]$ is a complex number, which we can denote as

```{math}
H[m] = A \cdot e^{\mathrm{j} \cdot \phi},
```
where $A \geq 0$ is the magnitude and $\phi$ is the phase.

Using the inverse DFT, we can interpret $X[m] = B \cdot e^{\mathrm{j} \cdot \theta}$ as encoding the magnitude $B$ and phase $\theta$ of a particular sinusoid present in the signal $x$.

Multiplying these two together, we see that $Y[m]$ has magnitude $A \cdot B$ and phase $\phi + \theta$.
That is, $H[m]$ encodes how much each sinusoidal component of $x$ is amplified (or attenuated) (by $A$) and delayed (by $\phi$) when convolved with $h$!

In the following sections, we'll see how to put this idea into practice for filtering signals.

### Dual convolution theorem

One interesting corollary of the convolution theorem is its so-called *dual* theorem:

````{admonition} Corollary: frequency-domain convolution

If $w$ and $x$ are sequences of length $N$, then element-wise multiplication in the time domain is equivalent to circular convolution *in the frequency domain*.

```{math}
\text{DFT}(w \cdot x) = \text{DFT}(w) * \text{DFT}(x)
```
````

The proof of this theorem is nearly identical to that of the convolution theorem, except that it uses a variation of the shifting theorem for the *inverse* DFT.

The dual convolution theorem is mainly useful as a theoretical device, as it can help us to understand the effects of element-wise multiplication (e.g., when windowing a signal).

We won't go into detail here, but the table below summarizes the relationships between DFT, IDFT, circular convolution, and element-wise multiplication.


| Time domain   |  | Frequency domain    |
| :--- | --- | ---: |
| Convolution $h*x$   | $\Leftrightarrow$ | Multiplication $H\cdot X$    |
| Multiplication $w\cdot x$   | $\Leftrightarrow$  | Convolution $W*X$    |
